## Análise a fim de orientar o tipo do próximo filme a ser desenvolvido em Hollywood

ML com aprendizagem em regressão, pois analisaremos dados passados e atuais, para decisão futura.

## Importar Bibliotecas e Carregar os Dados

In [189]:
import pandas as pd



In [190]:
series = pd.read_csv("./series_imdb.csv")

series.head()

,id,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,5,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"


## 2 Exploração e formatação de dados

### 2.1 Exploração

In [191]:
series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    object 
 3   Certificate    898 non-null    object 
 4   Runtime        999 non-null    object 
 5   Genre          999 non-null    object 
 6   IMDB_Rating    999 non-null    float64
 7   Overview       999 non-null    object 
 8   Meta_score     842 non-null    float64
 9   Director       999 non-null    object 
 10  Star1          999 non-null    object 
 11  Star2          999 non-null    object 
 12  Star3          999 non-null    object 
 13  Star4          999 non-null    object 
 14  No_of_Votes    999 non-null    int64  
 15  Gross          830 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 125.0+ KB


Nota-se a coluna responsável pelo ID, não identificada.

Também é possível ver que as colunas Runtime, Released_year e Gross estão sendo carregadas com tipo não correspondente.

- Runtime deveria ser int64 mas está como object, provavelmente por incluir "min" de minutos.
- Released_year como são apenas números deveria ser carregado, também, como int64. 
- Gross, em pesquisa, é o faturamento apenas no EUA e Canadá, portanto é um valor que representa dólares, assim deveria ser carregado como int64 por não considerar flutuação de centavos. 

In [192]:
series.isnull().sum()

id                 0
Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

Tabelas como id, Series_Title e Overview possuem baixa influência, portanto não são necessárias. Series_title é apenas o título do filme e Overview se torna inútil com a existência da coluna Genre.

Além disso, existem poucas linhas e colunas com valores nulos, portanto é possível limpá-los.

In [193]:
series['Certificate'].value_counts()

Certificate
U           234
A           196
UA          175
R           146
PG-13        43
PG           37
Passed       34
G            12
Approved     11
TV-PG         3
GP            2
TV-14         1
Unrated       1
TV-MA         1
16            1
U/A           1
Name: count, dtype: int64

É possível destacar a falta de especifidade devido a diferença de certificações por países, como é possível destacar em artigo publicado pelo IMDB - https://help.imdb.com/article/contribution/titles/certificates/GU757M8ZJ9ZPXB39?ref_=helpms_helpart_inline&utm_source=chatgpt.com#canada

Para resolver o problema é necessário adicionar uma coluna de severidade para a certificação, que ocorre de forma diferente para cada país mas possui a mesma lógica.

Com isso, é possível criar uma coluna com o nível de severidade em escala 
de 0-10, abrangindo categorias como 12 e 12+ com severidade 
igual a 3 e 4 (exemplo) e equivalências entre países.

Issues List
- Corrigir coluna "unnamed" para "id". (alteração é necessária ser feita no arquivo .csv para identificação na formatação)
- Limpar as colunas id, Series_Title e Overview
- Eliminar linhas com valores nulos.
- Corrigir os tipos das colunas Runtime, Released_year e Gross
- Adicionar nova coluna de severidade para a certificação.
- Converter as colunas para snake_case (boa prática)

### 2.2 Formatação dos dados

- Converter as colunas é um bom início

In [194]:
series.rename(lambda x: x.lower().strip().replace(" ", "_"), axis='columns', inplace=True)

series.columns

Index(['id', 'series_title', 'released_year', 'certificate', 'runtime',
       'genre', 'imdb_rating', 'overview', 'meta_score', 'director', 'star1',
       'star2', 'star3', 'star4', 'no_of_votes', 'gross'],
      dtype='object')

- Eliminar as linhas que possuem valores nulos

In [195]:
series = series.dropna()
series.isnull().sum()

id               0
series_title     0
released_year    0
certificate      0
runtime          0
genre            0
imdb_rating      0
overview         0
meta_score       0
director         0
star1            0
star2            0
star3            0
star4            0
no_of_votes      0
gross            0
dtype: int64

- Eliminar colunas desnecessárias

In [196]:
to_drop = ['id', 'series_title', 'overview']
series.drop(to_drop, axis='columns', inplace=True)

Afirmando o drop correto

In [197]:

assert series.columns.all() not in to_drop

- Corrigir os tipos das colunas Runtime, Released_year e Gross

Convertendo tipo do **Runtime** para "int64".

In [198]:
series['runtime'] = pd.to_numeric(series['runtime'].str.replace(' min', ''), errors='coerce').astype('Int64')

Confirmando tipo correto para a coluna

In [199]:
series.runtime.dtypes

Int64Dtype()

- Convertendo o tipo de **gross** para int64.

In [200]:
series['gross'] = pd.to_numeric(series['gross'].str.replace('$', '').str.replace(',', ''), errors='coerce').astype('Int64')

Confirmando

In [201]:
series.gross.dtypes

Int64Dtype()

- Convertendo o tipo de **released_year** para int64.

In [202]:
series['released_year'] = pd.to_numeric(series['released_year'], errors='coerce').astype('Int64')

Confirmando: 

In [203]:
series.released_year.dtypes

Int64Dtype()

- Última etapa é otimizar a coluna de certificações para nível de severidade 0-10

Começamos entendendo quais certificações devemos otimizar

In [208]:
series['certificate'].value_counts()

certificate
U           183
A           173
UA          142
R           131
PG-13        38
PG           19
G             9
Passed        9
Approved      6
TV-PG         1
U/A           1
GP            1
Name: count, dtype: int64

Baseado nas certificações do imdb anteriormente mencionada, podemos mapear numericamente.

In [210]:
mapeamento_severidade = {
    'G': 1,        # Audiência Geral
    'U': 1,        # Universal 
    'Passed': 2,   # Antigo sistema
    'Approved': 2, # Antigo sistema  
    'PG': 3,       # Parental Guidance
    'GP': 3,       # General/Parental
    'TV-PG': 4,    # TV Parental
    'PG-13': 5,    # 13+ anos
    'UA': 6,       # Unrestricted Adult
    'U/A': 6,      # Universal/Adult
    'R': 8,        # Restricted (mais severo)
    'A': 9         # Adults Only (máximo)
}

series['mapeamento_severidade'] = series['certificate'].map(mapeamento_severidade).astype('Int64')

Confirmando:

In [214]:
assert series['mapeamento_severidade'].isnull().sum() == 0

In [215]:
series.columns

Index(['released_year', 'certificate', 'runtime', 'genre', 'imdb_rating',
       'meta_score', 'director', 'star1', 'star2', 'star3', 'star4',
       'no_of_votes', 'gross', 'mapeamento_severidade'],
      dtype='object')

## 3. Univariate Exploration

Com os dados formatados e limpos, é possível analisar estatisticamente.